# Tutorial: Differential Privacy Evolution Strategy (DP-ES)

This notebook demonstrates how to use **DP-ES** (Differential Privacy Evolution Strategy) with TextGrad to optimize text variables while preserving privacy.

## 📚 What is DP-ES?

DP-ES extends TextGrad's optimization with differential privacy guarantees by:
- Maintaining a **population of candidate solutions** instead of a single variable
- Applying **DP-aware scoring** with clipping and noise injection
- Using **DP selection mechanisms** (Noisy Top-k, Exponential Mechanism)
- Tracking **privacy budget** (ε, δ) consumption using privacy accountants

## 🎯 When to use DP-ES?

Use DP-ES when:
- Your evaluation function uses **sensitive/private data**
- You need **provable privacy guarantees** (ε, δ-DP)
- You want to prevent **memorization** of training examples
- You're optimizing with **user data** or **proprietary information**

## 🔧 Setup

In [ ]:
# Import required libraries
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import List

import dp_textgrad as tg
from dp_textgrad import (
    Variable,
    DPEvolutionStrategy,
    DPEvolutionConfig,
    PrivacyAccountant,
    AdvancedCompositionAccountant,
    DPScorer,
    DPScorerConfig,
    DPSelector,
    DPSelectorConfig,
    MutationEngine,
    MutationConfig,
)

# Set your API key
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Set the backward engine for gradient computation
tg.set_backward_engine("gpt-4o-mini", override=True)

## 📝 Example 1: Basic DP-ES Prompt Optimization

Let's optimize a prompt for a simple math reasoning task with differential privacy.

In [ ]:
# Define the task: simple arithmetic reasoning
test_questions = [
    "What is 15 + 27?",
    "Calculate 43 - 18",
    "What is 12 * 8?",
    "Divide 144 by 12",
]

correct_answers = ["42", "25", "96", "12"]

# Initial prompt (suboptimal)
initial_prompt = "Solve this math problem."

print(f"Initial prompt: '{initial_prompt}'")

In [ ]:
# Create the target variable to optimize
prompt_variable = Variable(
    value=initial_prompt,
    role_description="instruction prompt for solving math problems",
    requires_grad=True
)

# Create a simple LLM model for evaluation
model = tg.BlackboxLLM("gpt-4o-mini")

# Define evaluation function (uses private test data)
def evaluate_prompt(variable: Variable) -> float:
    """Evaluate prompt on test questions (this uses private data!)."""
    prompt = variable.get_value()
    correct = 0
    
    for question, answer in zip(test_questions, correct_answers):
        response = model(Variable(f"{prompt}\n\nQuestion: {question}", role_description="query"))
        if answer in response.value:
            correct += 1
    
    # Return accuracy as score (higher is better)
    return correct / len(test_questions)

### Configure DP-ES Components

In [ ]:
# 1. Configure DP Scorer (handles clipping and noise)
scorer_config = DPScorerConfig(
    clipping_value=1.0,          # Clip scores to [-1, 1] range
    noise_multiplier=None,        # Auto-calibrate noise based on epsilon
    epsilon=0.5,                  # Privacy budget per scoring round
    delta=1e-5                    # Privacy parameter delta
)
dp_scorer = DPScorer(scorer_config)

# 2. Configure DP Selector (chooses parents with privacy)
selector_config = DPSelectorConfig(
    select_k=3,                   # Select top-3 candidates
    epsilon=0.1,                  # Privacy budget for selection
    sensitivity=1.0               # Sensitivity of the selection function
)
dp_selector = DPSelector(selector_config)

# 3. Configure Mutation Engine (generates new candidates)
def mutation_function(parent_candidate, iteration: int, rng: random.Random, feedback):
    """Generate variations of the parent prompt."""
    parent_prompt = parent_candidate.variable.get_value()
    
    # Use LLM to generate variations
    mutation_instruction = Variable(
        f"""Generate 2 improved variations of this math problem instruction.
        
Original instruction: "{parent_prompt}"

Make the variations:
- More clear and specific
- Include step-by-step guidance
- Encourage showing work

Return ONLY the two variations, one per line.""",
        role_description="mutation instruction"
    )
    
    response = model(mutation_instruction)
    variations = [line.strip() for line in response.value.split('\n') if line.strip()]
    
    # Create Variable objects for each variation
    children = []
    for var_text in variations[:2]:  # Take first 2 variations
        child = Variable(
            value=var_text,
            role_description=parent_candidate.variable.get_role_description(),
            requires_grad=True
        )
        children.append(child)
    
    return children

mutation_config = MutationConfig(
    offspring_per_parent=2,       # Generate 2 children per parent
    allow_identity_offspring=True # Allow keeping parent unchanged
)
mutation_engine = MutationEngine(
    mutation_fn=mutation_function,
    config=mutation_config
)

# 4. Configure Privacy Accountant (tracks privacy budget)
accountant = AdvancedCompositionAccountant(
    target_epsilon=3.0,           # Total privacy budget
    target_delta=1e-4,            # Total delta
    delta_slack=1e-6              # Slack for advanced composition
)

print("✓ All DP-ES components configured!")

### Run DP-ES Optimization

In [ ]:
# Create DP Evolution Strategy
evolution_config = DPEvolutionConfig(
    population_size=6,            # Maintain 6 candidates
    parents_to_select=3,          # Select 3 best as parents
    max_iterations=5,             # Run for 5 generations
    stop_on_budget=True,          # Stop if privacy budget exhausted
    rng_seed=42                   # For reproducibility
)

dp_optimizer = DPEvolutionStrategy(
    parameter=prompt_variable,
    evaluation_fn=evaluate_prompt,
    scorer=dp_scorer,
    selector=dp_selector,
    mutation_engine=mutation_engine,
    accountant=accountant,
    config=evolution_config
)

print("Starting DP-ES optimization...")
print(f"Privacy budget: ε={accountant.target_epsilon}, δ={accountant.target_delta}")
print("="*60)

# Run optimization
dp_optimizer.step()

print("\n" + "="*60)
print("Optimization complete!")
print(f"\nFinal optimized prompt: '{prompt_variable.get_value()}'")
print(f"\nPrivacy consumed: ε={accountant.consumed_epsilon:.3f}, δ={accountant.consumed_delta:.2e}")
print(f"Privacy remaining: ε={accountant.remaining_budget()[0]:.3f}, δ={accountant.remaining_budget()[1]:.2e}")

## 📊 Example 2: Visualizing Privacy-Performance Tradeoff

Let's compare different privacy budgets and see their impact on performance.

In [ ]:
# Define a simpler evaluation for faster iteration
def simple_evaluate(variable: Variable) -> float:
    """Simple scoring based on prompt length and keywords."""
    prompt = variable.get_value().lower()
    score = 0.0
    
    # Reward certain keywords
    if "step" in prompt:
        score += 0.3
    if "explain" in prompt or "show" in prompt:
        score += 0.3
    if "work" in prompt or "reasoning" in prompt:
        score += 0.2
    if len(prompt) > 20 and len(prompt) < 100:
        score += 0.2
    
    return min(score, 1.0)

# Test different epsilon values
epsilon_values = [0.1, 0.5, 1.0, 2.0, 5.0]
results = []

for eps in epsilon_values:
    print(f"\nTesting ε={eps}...")
    
    # Reset prompt
    test_prompt = Variable(
        value="Solve this.",
        role_description="instruction prompt",
        requires_grad=True
    )
    
    # Configure with current epsilon
    test_scorer = DPScorer(DPScorerConfig(
        clipping_value=1.0,
        noise_multiplier=None,
        epsilon=eps,
        delta=1e-5
    ))
    
    test_selector = DPSelector(DPSelectorConfig(
        select_k=2,
        epsilon=0.0,  # No additional epsilon for selection
        gumbel_scale_override=0.0
    ))
    
    # Simplified mutation
    def simple_mutation(parent, iteration, rng, feedback):
        base = parent.variable.get_value()
        variations = [
            base + " Show your work.",
            base + " Explain step by step."
        ]
        return [Variable(v, role_description="prompt", requires_grad=True) for v in variations]
    
    test_mutation = MutationEngine(
        mutation_fn=simple_mutation,
        config=MutationConfig(offspring_per_parent=2)
    )
    
    test_accountant = PrivacyAccountant(target_epsilon=10.0, target_delta=1e-4)
    
    optimizer = DPEvolutionStrategy(
        parameter=test_prompt,
        evaluation_fn=simple_evaluate,
        scorer=test_scorer,
        selector=test_selector,
        mutation_engine=test_mutation,
        accountant=test_accountant,
        config=DPEvolutionConfig(
            population_size=4,
            parents_to_select=2,
            max_iterations=3,
            rng_seed=42
        )
    )
    
    optimizer.step()
    
    final_score = simple_evaluate(test_prompt)
    results.append((eps, final_score, test_accountant.consumed_epsilon))
    print(f"  Final score: {final_score:.3f}, ε consumed: {test_accountant.consumed_epsilon:.3f}")

In [ ]:
# Plot results
epsilons = [r[0] for r in results]
scores = [r[1] for r in results]

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(epsilons, scores, 'o-', linewidth=2, markersize=8)
plt.xlabel('Privacy Budget (ε per iteration)', fontsize=12)
plt.ylabel('Final Score', fontsize=12)
plt.title('Privacy-Performance Tradeoff', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xscale('log')

plt.subplot(1, 2, 2)
total_epsilon = [r[2] for r in results]
plt.bar(range(len(epsilons)), total_epsilon, color='steelblue', alpha=0.7)
plt.xlabel('Configuration Index', fontsize=12)
plt.ylabel('Total ε Consumed', fontsize=12)
plt.title('Privacy Budget Consumption', fontsize=14, fontweight='bold')
plt.xticks(range(len(epsilons)), [f'ε={e}' for e in epsilons], rotation=45)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Key Insight:")
print("Higher ε (less privacy) → Better performance but less privacy protection")
print("Lower ε (more privacy) → May need more iterations or larger population")

## 🔍 Example 3: Understanding Privacy Components

Let's break down what each component does.

In [ ]:
# Demonstrate clipping and noise
from dp_textgrad.dp_es.population import Candidate

print("=" * 60)
print("DEMONSTRATION: Score Clipping and Noise Injection")
print("=" * 60)

# Create test candidates
test_var = Variable("test", role_description="test", requires_grad=False)
candidates = [Candidate(test_var, {"id": i}) for i in range(5)]

# Raw scores (could leak information!)
raw_scores = [0.2, 0.5, 0.85, 1.2, 2.5]  # Note: some exceed clipping bound

# Create scorer with clipping
demo_scorer = DPScorer(DPScorerConfig(
    clipping_value=1.0,
    noise_multiplier=0.5,
    epsilon=0.5,
    delta=1e-5
))

# Simulate scoring
def eval_fn(cand):
    idx = int(cand.metadata["id"])
    return raw_scores[idx]

dp_scores = demo_scorer.evaluate(candidates, eval_fn, rng=random.Random(42))

# Display results
print("\nCandidate | Raw Score | Clipped | DP Score (with noise)")
print("-" * 60)
for i, record in enumerate(dp_scores.records):
    print(f"    {i}     |   {record.raw_score:.2f}    |  {record.clipped_score:.2f}  |      {record.dp_score:.2f}")

print("\n💡 Notice:")
print("1. Scores > 1.0 are clipped to 1.0")
print("2. Gaussian noise is added to clipped scores")
print("3. This prevents exact score leakage!")
print(f"\nPrivacy cost: ε={dp_scores.epsilon}, δ={dp_scores.delta}")

## 📖 Key Parameters Guide

### DPScorerConfig
- `clipping_value`: Bound for score clipping (controls sensitivity)
- `noise_multiplier`: Scale of Gaussian noise (if None, auto-calibrated from ε)
- `epsilon`: Privacy budget per scoring call
- `delta`: Privacy parameter (typically ~1/n²)

### DPSelectorConfig
- `select_k`: Number of parents to select
- `epsilon`: Privacy budget for selection (0 = deterministic on noisy scores)
- `sensitivity`: Sensitivity of selection function

### DPEvolutionConfig
- `population_size`: Number of candidates to maintain
- `parents_to_select`: How many candidates become parents
- `max_iterations`: Maximum evolution rounds
- `stop_on_budget`: Stop early if privacy budget exhausted

### Privacy Accountant
- `PrivacyAccountant`: Basic sequential composition (ε_total = Σε_i)
- `AdvancedCompositionAccountant`: Uses advanced composition theorem (tighter bounds!)

## 🎓 Best Practices

### 1. Choosing ε (Epsilon)
- **ε < 1**: Strong privacy (research/healthcare)
- **1 ≤ ε ≤ 3**: Moderate privacy (general use)
- **ε > 3**: Weak privacy (only mild protection)

### 2. Setting Population Size
- Larger population → Better exploration but more privacy cost
- Typical: 4-10 candidates
- Increase if privacy budget allows

### 3. Clipping Value
- Should match expected score range
- Too small → Information loss
- Too large → Excessive noise
- Monitor `clipped_score` vs `raw_score` ratio

### 4. Noise Calibration
- Use `noise_multiplier=None` for automatic calibration
- Manual tuning: σ = clipping_value × noise_multiplier

### 5. Privacy Budget Allocation
- Split budget across iterations: ε_per_iter = ε_total / max_iterations
- Use AdvancedCompositionAccountant for better bounds
- Monitor remaining budget with `accountant.remaining_budget()`

## 🚀 Next Steps

1. **Try on your own task**: Adapt the evaluation function to your use case
2. **Experiment with parameters**: Test different ε values and population sizes
3. **Add critique pipeline**: Use `CritiquePipeline` for more sophisticated mutations
4. **Compare with TGD**: See `Tutorial-DP-ES-Comparison.ipynb` for side-by-side comparison
5. **Read the paper**: Check `DP-TextGrad via DP-ES.md` for theoretical background

## 📚 Additional Resources

- [Differential Privacy Primer](https://programming-dp.com/)
- [TextGrad Documentation](https://dp_textgrad.readthedocs.io/)
- [Evolution Strategies Overview](https://blog.otoro.net/2017/10/29/visual-evolution-strategies/)